<a href="https://colab.research.google.com/github/vifirsanova/W2V/blob/main/Firsanova_W2V_zachet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Автор: [Виктория Фирсанова](https://github.com/vifirsanova)

# Загрузка библиотек

In [1]:
# библиотека для цветного вывода
!pip install colorama

In [2]:
# PoS-разметка
!pip install natasha

In [3]:
from colorama import Fore, Style # цветной вывод
import json # работа с javascript object notation
import random # случайное перемешивание

from natasha import NewsMorphTagger, NewsEmbedding, Doc, Segmenter #PoS-разметка

# Загрузка корпуса

* [Корпус новостей газеты Московский комсомолец за 2020-2021 года.](https://github.com/vifirsanova/W2V/blob/main/corpus.json) 

* Собран автором в рамках семинарских занятий.

* Корпус загружен на GitHub с помощью терминала git с помощью следующего кода:

```
git clone git@github.com:vifirsanova/W2V.git
cd w2v
git add corpus.json
git commit -m "Add file"
git push
```



In [4]:
# подгружаем корпус из GitHub
!wget https://raw.githubusercontent.com/vifirsanova/W2V/main/corpus.json

--2021-12-18 18:39:36--  https://raw.githubusercontent.com/vifirsanova/W2V/main/corpus.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92111843 (88M) [text/plain]
Saving to: ‘corpus.json’

corpus.json         100%[===================>]  87.84M   215MB/s    in 0.4s    

2021-12-18 18:39:39 (215 MB/s) - ‘corpus.json’ saved [92111843/92111843]



In [5]:
# сохраняем корпус в переменную
with open('corpus.json', encoding='utf-8', newline='') as json_file:
    data = json.load(json_file)

In [6]:
# образец данных корпуса
print(f"{Fore.RED}Образец данных корпуса:\n")
data['data'][0]

Образец данных корпуса:



{'date': '31.03.2020',
 'link': 'https://www.mk.ru/social/2020/03/31/minobrnauki-pereneslo-vstupitelnye-ekzameny-v-vuzy-izza-koronavirusa.html',
 'rubric': 'Общество',
 'text': '\nВ Минобрнауки РФ приняли решение перенести даты вступительных экзаменов в вузы из-за коронавируса, заявил глава ведомства Валерий Фальков в интервью «Россия 24».\nОн отметил, что в этом году число бюджетных мест было увеличено на 34 тысячи. Сейчас начался экономический спад, но менять стратегию в министерстве не намерены.\nРешение об увеличении бюджетных мест в вузах связано с ростом числа выпускников школ. Будет увеличено число мест на педагогическом и медицинском направлениях. Большой запрос на бюджетные места у регионов, подчеркнул Фальков.\nРанее власти перенесли даты ЕГЭ и ОГЭ.\nЧитайте также:\xa0Скворцова похвалила российский препарат от коронавируса\n\xa0\n\n',
 'time': '09:11',
 'title': 'Минобрнауки решило перенести вступительные экзамены в вузы из-за коронавируса'}

In [7]:
# собираем тексты из корпуса
full_dataset = []

for elem in data['data']:
    full_dataset.append(elem['text'])

# случайное перемешивание данных
random.shuffle(full_dataset)

# деление на обучающую (80% данных) и проверочную (20% данных) выборку
TRAIN_VAL_SPLIT = int(len(full_dataset) * 0.8)
train = full_dataset[:TRAIN_VAL_SPLIT]
val = full_dataset[TRAIN_VAL_SPLIT:]

print(f"{Fore.RED}Образец текста из выборки:{Style.RESET_ALL}\n", ''.join(train[0]))
print(f"{Fore.RED}Размер обучающей выборки:{Style.RESET_ALL} {len(train)} новостей, {sum([len(x.split()) for x in train])} слов, {sum([len(x) for x in train])} символов\
\n{Fore.RED}Размер проверочной выборки:{Style.RESET_ALL} {len(val)} новостей, {sum([len(x.split()) for x in val])} слов, {sum([len(x) for x in val])} символов")

Образец текста из выборки:
 
Исполнительный директор по перспективным программам и науке "Роскосмоса" Александр Блошенко заявил, что госкорпорация отказалась от участия США в проекте по запуску на Венеру межпланетной станции "Венера-Д".
Также он отметил, что обнаружение биомаркеров в атмосфере соседней с Землей планеты не может считаться объективным доказательством наличия жизни там. Достоверные научные данные могут быть получены только при контактном исследовании планеты.
Между тем ближайшая миссия на Венеру запланирована на 2027-2029 годы. Проект пока не получил финансирования с российской стороны. При этом ученые просят на реализацию российской части проекта более 17 миллиардов рублей на ближайшие 10 лет.


Размер обучающей выборки: 36736 новостей, 3884868 слов, 37325192 символов
Размер проверочной выборки: 9184 новостей, 965140 слов, 9222992 символов


# Разметка

Для частеречной разметки можно прописать собственный парсер либо использовать готовый. 

[Образец собственного парсера](https://colab.research.google.com/drive/19ZDb0FR6wO2zA26vYzJMf8xFkFqwBq72?usp=sharing). Работает на базе ConvNet.

Здесь для разметки используется библиотека [Natasha](https://github.com/natasha/natasha/tree/61d44032ca10612bd2b37444cdb50077402dd386).

In [49]:
# преобразуем тексты из обучающей и проверочной выборок к виду объектов, подходящих для обработки с помощью библиотеки Natasha 
train_doc, val_doc = [Doc(x) for x in train], [Doc(x) for x in val]
print(f"{Fore.RED}Образец обработанного текста:{Style.RESET_ALL}", val_doc[0])

# инструмент для сегментации текстов
segmenter = Segmenter()
# сегментация обучающей выборки
for x in train_doc:
  x.segment(segmenter)
# сегментация проверочной выборки
for x in val_doc:
  x.segment(segmenter)

print(f"{Fore.RED}Образец сегментированного текста:{Style.RESET_ALL}", val_doc[5].tokens[0])

Образец обработанного текста: Doc(text='\nРоссийская «Партия пенсионеров» произвела расче...)
Образец сегментированного текста: DocToken(start=1, stop=16, text='Премьер-министр')


In [61]:
# PoS-разметка
morph_tagger = NewsMorphTagger(NewsEmbedding())
# разметка обучающей выборки
for x in train_doc:
  x.tag_morph(morph_tagger)

# разметка проверочной выборки
for x in val_doc:
  x.tag_morph(morph_tagger)

print(f"{Fore.RED}Образец размеченного текста:{Style.RESET_ALL}", val_doc[5].tokens[0])

Образец размеченного текста: DocToken(start=1, stop=16, text='Премьер-министр', pos='NOUN', feats=<Anim,Nom,Masc,Sing>)


In [99]:
# сохраним полученные данные для дальнейшего использования
# разметка обучающей выборки
# создаем словарь, где будут храниться размеченные данные
train_corp = {}
# для каждого текста обучающей выборки
for n in range(len(train_doc)):
  # сохраняем в словарь PoS-тег и морфологические характеристики слова
  for k in range(len(train_doc[n].tokens)):
    train_corp[train_doc[n].tokens[k].text] = [train_doc[n].tokens[k].pos, train_doc[n].tokens[k].feats]

# разметка проверочной выборки
# создаем словарь, где будут храниться размеченные данные
val_corp = {}
# для каждого текста проверочной выборки
for n in range(len(val_doc)):
  # сохраняем в словарь PoS-тег и морфологические характеристики слова
  for k in range(len(val_doc[n].tokens)):
    val_corp[val_doc[n].tokens[k].text] = [val_doc[n].tokens[k].pos, val_doc[n].tokens[k].feats]

print(f"{Fore.RED}Образец размеченного корпуса:{Style.RESET_ALL}", list(val_corp.items())[0])
print(f"{Fore.RED}Объем размеченного корпуса обучающей выборки:{Style.RESET_ALL}", len(train_corp))
print(f"{Fore.RED}Объем размеченного корпуса проверочной выборки:{Style.RESET_ALL}", len(val_corp))

Образец размеченного корпуса: ('Российская', ['ADJ', {'Case': 'Nom', 'Degree': 'Pos', 'Gender': 'Fem', 'Number': 'Sing'}])
Объем размеченного корпуса обучающей выборки: 194057
Объем размеченного корпуса проверочной выборки: 92561


In [104]:
# сохраним размеченный корпус в отдельный файл, чтобы его можно было использовать повторно
with open('train_corp.json', 'w') as f:
    json.dump(train_corp, f, ensure_ascii=False)

with open('val_corp.json', 'w') as f:
    json.dump(val_corp, f, ensure_ascii=False)